In [9]:
import numpy as np

def find_float32_max(r):
    tmp = 0.0
    for value in r:
        if value > tmp:
            tmp = value
    return tmp

def find_float32_min(r):
    tmp = r[0]
    for value in r:
        if value < tmp:
            tmp = value
    return tmp

def s_value(r, q):
    r_max = find_float32_max(r)
    r_min = find_float32_min(r)
    q_max = 127
    q_min = -128

    s = (r_max - r_min) / (q_max - q_min)
    return s

def z_value(r, q):
    r_max = find_float32_max(r)
    q_max = 127
    s = s_value(r, q)
    z = round(q_max - r_max / s)  # May be out of bounds
    return z

def float32_to_int8(r):
    s = [0.0]
    z = [0.0]
    q =np.zeros(64)
    '''q = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
         0, 0, 0, 0]  # Initialize q with zeros'''
    s[0] = s_value(r, q)
    z[0] = z_value(r, q)

    print(f"s={s[0]}")
    print(f"z={z[0]}")

    for i in range(len(r)):
        q[i] = int(round(r[i] / s[0] + z[0]))
    return q,s,z
################### float32数据转int8模块  ##############

def float32_to_int88(float32_array): 
   # 定义量化范围（最小和最大值）
    min_val = np.min(float32_array)
    max_val = np.max(float32_array)

    # 计算缩放因子和零点
    scale = (max_val - min_val) / 127.0  # 127 是有符号 int8 的最大值
    zero_point = np.round(0 - min_val / scale)

    # 使用缩放因子和零点进行量化
    int8_array = np.round(float32_array / scale + zero_point).astype(np.int8)
    print(f"scale={scale}")
    print(f"zero_point={zero_point}")
    return int8_array,scale, zero_point   

def float32_to_int8_test():
    r = [ 0.22131348,  0.0209198 ,  0.40454102,  0.31542969 ,-1.00585938 , 0.79296875,
 -0.64404297, -0.32324219  ,0.27709961 , 0.15075684, -1.38867188 , 0.02134705,
  0.47875977 , 0.07434082, -0.54541016,  0.30371094, -0.22265625, 0.37109375,
 -0.21887207,  1.09863281 ,-0.22363281, -0.48706055, -0.03918457, -0.40722656,
 -0.49291992,  0.40966797,  0.40795898, -0.56640625, -0.3190918,  -0.65966797,
 -0.31225586, -0.13891602,  0.66162109,  0.15551758, -0.40356445, -0.20556641,
 -0.2199707,   0.13305664, -0.26953125,  0.30834961,  0.32275391, -0.86523438,
 -0.40283203,  0.26098633,  0.34155273, -0.15808105, -0.14147949,  0.57958984,
 -0.63867188, -0.23388672,  0.19836426, -0.46459961, -0.15612793,  0.54296875,
  1.23339844,  0.84960938, -0.89990234,  0.92919922,-0.46948242 , 0.19482422,
 -0.08013916,  1.9921875 ,  0.10705566,  0.35058594]
    rr = [ 0.22131348,  0.0209198 ,  0.40454102,  0.31542969 ,-1.00585938 , 0.79296875,
 -0.64404297, -0.32324219  ,0.27709961 , 0.15075684, -1.38867188 , 0.02134705,
  0.47875977 , 0.07434082, -0.54541016,  0.30371094, -0.22265625, 0.37109375,
 -0.21887207,  1.09863281 ,-0.22363281, -0.48706055, -0.03918457, -0.40722656,
 -0.49291992,  0.40966797,  0.40795898, -0.56640625, -0.3190918,  -0.65966797,
 -0.31225586, -0.13891602,  0.66162109,  0.15551758, -0.40356445, -0.20556641,
 -0.2199707,   0.13305664, -0.26953125,  0.30834961,  0.32275391, -0.86523438,
 -0.40283203,  0.26098633,  0.34155273, -0.15808105, -0.14147949,  0.57958984,
 -0.63867188, -0.23388672,  0.19836426, -0.46459961, -0.15612793,  0.54296875,
  1.23339844,  0.84960938, -0.89990234,  0.92919922,-0.46948242 , 0.19482422,
 -0.08013916,  1.9921875 ,  0.10705566,  0.35058594]
    
    (q,s,z) = float32_to_int8(r)
    
    (int8_array,scale,zero_point) = float32_to_int88(r)

    print("quantification result:")
    for i in range(len(r)):
        print("q[{}] = {}".format(i, q[i]), end=' ')
        print("int8_array[{}] = {}".format(i, int8_array[i]), end=' ')
    print()

    print("inverse quantization result:")
    for i in range(len(r)):
        r[i] = s[0] * (q[i] - z[0])
        rr[i] = scale * (int8_array[i] - zero_point)
        print("r[{}] = {}".format(i, r[i]), end=' ')
        print("rr[{}] = {}".format(i, rr[i]), end=' ')
    print()

if __name__ == "__main__":
    float32_to_int8_test()


s=0.013258272078431372
z=-23
scale=0.02662094
zero_point=52.0
quantification result:
q[0] = -6.0 int8_array[0] = 60 q[1] = -21.0 int8_array[1] = 53 q[2] = 8.0 int8_array[2] = 67 q[3] = 1.0 int8_array[3] = 64 q[4] = -99.0 int8_array[4] = 14 q[5] = 37.0 int8_array[5] = 82 q[6] = -72.0 int8_array[6] = 28 q[7] = -47.0 int8_array[7] = 40 q[8] = -2.0 int8_array[8] = 62 q[9] = -12.0 int8_array[9] = 58 q[10] = -128.0 int8_array[10] = 0 q[11] = -21.0 int8_array[11] = 53 q[12] = 13.0 int8_array[12] = 70 q[13] = -17.0 int8_array[13] = 55 q[14] = -64.0 int8_array[14] = 32 q[15] = 0.0 int8_array[15] = 63 q[16] = -40.0 int8_array[16] = 44 q[17] = 5.0 int8_array[17] = 66 q[18] = -40.0 int8_array[18] = 44 q[19] = 60.0 int8_array[19] = 93 q[20] = -40.0 int8_array[20] = 44 q[21] = -60.0 int8_array[21] = 34 q[22] = -26.0 int8_array[22] = 51 q[23] = -54.0 int8_array[23] = 37 q[24] = -60.0 int8_array[24] = 33 q[25] = 8.0 int8_array[25] = 67 q[26] = 8.0 int8_array[26] = 67 q[27] = -66.0 int8_array[27] = 31 

In [ ]:
import numpy as np

def quantize_float32_to_int8(float32_array):
    # 定义量化范围（最小和最大值）
    min_val = np.min(float32_array)
    max_val = np.max(float32_array)

    # 计算缩放因子和零点
    scale = (max_val - min_val) / 127.0  # 127 是有符号 int8 的最大值
    zero_point = np.round(0 - min_val / scale)

    # 使用缩放因子和零点进行量化
    int8_array = np.round(float32_array / scale + zero_point).astype(np.int8)

    return int8_array, scale, zero_point

# 示例用法
if __name__ == "__main__":
    # 创建一个示例的浮点数数组
    float32_array = np.array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=np.float32)

    # 将浮点数数组量化为有符号 int8 数组
    int8_array, scale, zero_point = quantize_float32_to_int8(float32_array)

    print("Float32 数组:")
    print(float32_array)
    print("有符号 Int8 数组:")
    print(int8_array)
    print("缩放因子:", scale)
    print("零点:", zero_point)
